In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd

%load_ext lab_black
%load_ext watermark

# Dental Development

This example demonstrates ...

Adapted from [unit 10: growth.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/growth.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

## Associated lecture video: Unit 10 Lesson 2

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=99" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement


Data set on dental development was first provided by Potthof and Roy in their 1964 paper. It consists of longitudinal observations on 11 girls (gender=1) and 16 boys (gender=2). 

There are 4 observations on each subject centered at times -3,-1, 1, 3, where the units were years.

The measurement on each subject is the distance (in mm) from the center of the pituitary to the pteryomaxillary fisure.

Potthoff and Roy (1964). "A Generalized Multivariate Analysis of Variance Model Useful Especially for Growth Curve Problems," Biometrika, 51, 313-326.

MVN with Gender Specific Means but Common Precision Matrix